# Driver Aggression Neural Network (DANN)

Driver Aggression Neural Network is assigning an aggression value to a sorted set of sensory data. Driving is simulated in BeamNG v0.27 using their BeamNGpy open-source library.

In [154]:
import pandas as pd

parquet_file_path = '../imu_data_2023_05_03_13_16_18.parquet'
data = pd.read_parquet(parquet_file_path)

## Modify training data

- Group together data recorded from the same sensor
- Take around 100-1000 recorded data without the aggression values
- Make aggression values the label of the dataset
- Create a lot of training data by chunking the sorted (by timestamp) records.

In [155]:
import numpy as np

# Assuming 'data' is your Scheme
# data = pd.DataFrame(columns=[
#     'imuId',
#     'vehicleAggression',
#     'time',
#     'pos',
#     'dirX',
#     'dirY',
#     'dirZ',
#     'angVel',
#     'angAccel',
#     'mass',
#     'accRaw',
#     'accSmooth'
# ])

# Function to split the data into chunks
def split_into_chunks(data, chunk_size):
    return [data[i:i+chunk_size] for i in range(0, len(data), chunk_size)]

# Group the data by 'imuId' and sort within each group by 'time'
grouped_data = data.groupby('imuId').apply(lambda x: x.sort_values('time')).reset_index(drop=True)

# Set the desired chunk size (number of records per chunk)
chunk_size = 200

# Split the data into chunks and assign the 'vehicleAggression' value as the label
training_data = []
for imu_id, group in grouped_data.groupby('imuId'):
    chunks = split_into_chunks(group, chunk_size)
    for chunk in chunks:
        if len(chunk) >= chunk_size:
            label = chunk['vehicleAggression'].iloc[0]
            first_timestamp = chunk['time'].iloc[0]
            adjusted_time = chunk['time'] - first_timestamp
            
            # Separate list columns into individual columns
            # pos_df = pd.DataFrame(chunk['pos'].tolist(), columns=['posX', 'posY', 'posZ'], index=chunk.index)
            dir_x_df = pd.DataFrame(chunk['dirX'].tolist(), columns=['dirXX', 'dirXY', 'dirXZ'], index=chunk.index)
            dir_y_df = pd.DataFrame(chunk['dirY'].tolist(), columns=['dirYX', 'dirYY', 'dirYZ'], index=chunk.index)
            dir_z_df = pd.DataFrame(chunk['dirZ'].tolist(), columns=['dirZX', 'dirZY', 'dirZZ'], index=chunk.index)
            acc_raw_df = pd.DataFrame(chunk['accRaw'].tolist(), columns=['accRawX', 'accRawY', 'accRawZ'], index=chunk.index)
            acc_smooth_df = pd.DataFrame(chunk['accSmooth'].tolist(), columns=['accSmoothX', 'accSmoothY', 'accSmoothZ'], index=chunk.index)
            ang_vel_df = pd.DataFrame(chunk['angVel'].tolist(), columns=['angVelX', 'angVelY', 'angVelZ'], index=chunk.index)
            ang_accel_df = pd.DataFrame(chunk['angAccel'].tolist(), columns=['angAccelX', 'angAccelY', 'angAccelZ'], index=chunk.index)
            
            expanded_chunk = pd.concat(
                [
                    chunk,
                    # pos_df,
                    dir_x_df,
                    dir_y_df,
                    dir_z_df,
                    acc_raw_df,
                    acc_smooth_df,
                    ang_vel_df,
                    ang_accel_df
                ],
                axis=1
            )
            
            updated_chunk = (
                expanded_chunk.assign(time=adjusted_time)
                .drop(['imuId', 'mass', 'vehicleAggression', 'pos', 'dirX', 'dirY', 'dirZ', 'angVel', 'angAccel', 'accRaw', 'accSmooth'], axis=1)
            )
            
            training_data.append({'data': updated_chunk, 'label': label})

# Convert the list of dictionaries to a DataFrame
training_data_df = pd.DataFrame(training_data)

# Example of a single training set
# Set display options
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", 100)  # Set the maximum column width to 100 characters
pd.set_option("display.expand_frame_repr", False)
print(training_data_df.loc[0, 'data'])
pd.reset_option("all")


        time     dirXX     dirXY     dirXZ     dirYX     dirYY     dirYZ     dirZX     dirZY     dirZZ    accRawX    accRawY    accRawZ  accSmoothX  accSmoothY  accSmoothZ   angVelX   angVelY   angVelZ  angAccelX  angAccelY  angAccelZ
0    0.00000 -0.708555 -0.703101  0.059992 -0.001492 -0.083523 -0.996505  0.705654 -0.706168  0.058132  -2.000044   0.455348   1.687304   -1.434216    0.180359    1.261864  0.019011 -0.009269 -0.006625  -1.862096   0.423417   6.167933
1    0.01025 -0.708555 -0.703101  0.059992 -0.001492 -0.083523 -0.996505  0.705654 -0.706168  0.058132  -0.694419  -0.174066   1.633643   -0.842604   -0.103073    1.559174  0.009397 -0.004004 -0.002393  -1.326108   1.985857  -0.100370
2    0.02050 -0.708593 -0.703074  0.059857 -0.001539 -0.083289 -0.996524  0.705615 -0.706223  0.057936  -0.248124  -0.637252  -5.505297   -0.367201   -0.530253   -4.090256  0.012850  0.000724 -0.009720   0.026034   0.057339  -5.679400
3    0.03075 -0.708820 -0.702860  0.059679 -0.001632 -0.0829

## Translating the training data to learn

In [156]:
from sklearn.model_selection import train_test_split

# Get the data and labels from the training_data_df
X = np.stack(training_data_df['data'].apply(lambda x: x.to_numpy()).to_numpy())
y = training_data_df['label'].to_numpy()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (1267, 200, 22)
X_test shape: (317, 200, 22)
y_train shape: (1267,)
y_test shape: (317,)


## Learn with PyTorch

- Create a TensorDataset
- Create a DataLoader, which shuffles the data
- Create a simple neural net (torch.nn.Sequential) which uses CUDA while training
- Train the neural net with the data provided
- Evaluate the net with the test data

In [157]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# # Pad sequences to the same length
# X_train_padded = pad_sequences(X_train, dtype='float32', padding='post')
# y_train_padded = pad_sequences(y_train, dtype='float32', padding='post')
# X_test_padded = pad_sequences(X_test, dtype='float32', padding='post')
# y_test_padded = pad_sequences(y_test, dtype='float32', padding='post')

# Create tensors from the padded data
X_train_tensor = torch.tensor(X_train).permute(0, 2, 1)
y_train_tensor = torch.tensor(y_train)
X_test_tensor = torch.tensor(X_test).permute(0, 2, 1)
y_test_tensor = torch.tensor(y_test)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [158]:
import torch.nn as nn

# Define the CNN architecture
class CNNRegressor(nn.Module):
    def __init__(self, input_channels, num_filters, kernel_size, pool_size, hidden_units, dropout_rate, device):
        super(CNNRegressor, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv1d(input_channels, num_filters, kernel_size),
            nn.BatchNorm1d(num_filters),
            nn.ReLU(),
            nn.MaxPool1d(pool_size)
        ).to(device)

        self.conv2 = nn.Sequential(
            nn.Conv1d(num_filters, num_filters * 2, kernel_size),
            nn.BatchNorm1d(num_filters * 2),
            nn.ReLU(),
            nn.MaxPool1d(pool_size)
        ).to(device)

        self.conv3 = nn.Sequential(
            nn.Conv1d(num_filters * 2, num_filters * 4, kernel_size),
            nn.BatchNorm1d(num_filters * 4),
            nn.ReLU(),
            nn.MaxPool1d(pool_size)
        ).to(device)

        self.flatten = nn.Flatten()

        conv1_out_size = (chunk_size - kernel_size + 1) // pool_size
        conv2_out_size = (conv1_out_size - kernel_size + 1) // pool_size
        conv3_out_size = (conv2_out_size - kernel_size + 1) // pool_size

        self.fc1 = nn.Sequential(
            nn.Linear(num_filters * 4 * conv3_out_size, hidden_units),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        ).to(device)

        self.fc2 = nn.Sequential(
            nn.Linear(hidden_units, hidden_units // 2),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        ).to(device)

        self.fc3 = nn.Linear(hidden_units // 2, 1).to(device)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [159]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_channels = X_train.shape[2]

In [160]:
import optuna
import random
import torch.optim as optim

def train_and_eval_net(trial):
    # Suggest hyperparameters using the trial object
    num_filters = trial.suggest_int("num_filters", 256, 512)
    kernel_size = trial.suggest_int("kernel_size", 3, 5)
    pool_size = trial.suggest_int("pool_size", 2, 4)
    hidden_units = trial.suggest_int("hidden_units", 24, 92)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    
    # Value: 0.029481003992259502
    # Params: {'num_filters': 454, 'kernel_size': 3, 'pool_size': 4, 'hidden_units': 53, 'dropout_rate': 0.15038840244397983, 'learning_rate': 5.2460887662900957e-05}
    # num_filters = 454
    # kernel_size = 3
    # pool_size = 4
    # hidden_units = 53
    # dropout_rate = 0.15038840244397983
    # learning_rate = 5.2460887662900957e-05]
    
    # Value: 0.028036039788275957
    # Params: {'num_filters': 437, 'kernel_size': 4, 'pool_size': 4, 'hidden_units': 73, 'dropout_rate': 0.1657660256105821, 'learning_rate': 3.964884325777312e-05}
    # num_filters = 437
    # kernel_size = 4
    # pool_size = 4
    # hidden_units = 73
    # dropout_rate = 0.1657660256105821
    # learning_rate = 3.964884325777312e-05
    
    # Create the model
    net = CNNRegressor(input_channels, num_filters, kernel_size, pool_size, hidden_units, dropout_rate, device)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    num_epochs = 50
    
    net.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.float().to(device), labels.float().to(device)
    
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
    
            running_loss += loss.item()
        # print(f"Epoch {epoch+1}/{num_epochs} Loss: {running_loss/len(train_loader)}")

    net.eval()
    with torch.no_grad():
        total_difference = 0.0
        fake_difference = 0.0
        num_samples = 0
        test_loss = 0.0
        for inputs, labels in test_loader:
            inputs, labels = inputs.float().to(device), labels.float().to(device)
            outputs = net(inputs)

            # Calculate the absolute difference between the predicted and real labels
            difference = torch.abs(outputs.squeeze() - labels)

            # Update the total difference and the number of samples
            total_difference += difference.sum().item()
            num_samples += len(labels)

            loss = criterion(outputs.squeeze(), labels)
            test_loss += loss.item()

            # random outputs
            fake_outputs = torch.tensor([random.uniform(0.2, 0.6) for _ in range(len(outputs))], device=device)
            fake_diff = torch.abs(fake_outputs - labels)
            fake_difference += fake_diff.sum().item()

        print(f"Test Loss: {test_loss/len(test_loader)}")

        # Calculate the average absolute difference
        average_difference = total_difference / num_samples
        average_fake_difference = fake_difference / num_samples
        print(f"Average Absolute Difference: {average_difference}")
        print(f"Average Absolute Fake Difference: {average_fake_difference}")
    
    return test_loss/len(test_loader)

# train_and_eval_net(None)

## Hyperparameter optimization via Optuna

In [161]:
study = optuna.create_study(direction="minimize")
study.optimize(train_and_eval_net, n_trials=20)  # You can adjust the number of trials depending on your computational resources

[I 2023-05-03 14:29:11,708] A new study created in memory with name: no-name-d82a1031-17fe-4aba-9d21-210045bf08bb
[I 2023-05-03 14:30:04,833] Trial 0 finished with value: 0.10289446761210759 and parameters: {'num_filters': 407, 'kernel_size': 3, 'pool_size': 2, 'hidden_units': 29, 'dropout_rate': 0.3719176929230378, 'learning_rate': 0.0008220190951594898}. Best is trial 0 with value: 0.10289446761210759.


Test Loss: 0.10289446761210759
Average Absolute Difference: 0.2682291746891635
Average Absolute Fake Difference: 0.23194245058655363


[I 2023-05-03 14:31:27,400] Trial 1 finished with value: 0.031783935924371086 and parameters: {'num_filters': 331, 'kernel_size': 4, 'pool_size': 2, 'hidden_units': 57, 'dropout_rate': 0.13682306138658817, 'learning_rate': 2.343593459453175e-05}. Best is trial 1 with value: 0.031783935924371086.


Test Loss: 0.031783935924371086
Average Absolute Difference: 0.14330925926425103
Average Absolute Fake Difference: 0.2210026148365875


[I 2023-05-03 14:33:24,733] Trial 2 finished with value: 0.03317089999715487 and parameters: {'num_filters': 485, 'kernel_size': 5, 'pool_size': 2, 'hidden_units': 91, 'dropout_rate': 0.21832811639189167, 'learning_rate': 1.705668841037158e-05}. Best is trial 1 with value: 0.031783935924371086.


...

Test Loss: 0.03176876219610373
Average Absolute Difference: 0.14066249414197277
Average Absolute Fake Difference: 0.228411713609184


[I 2023-05-03 14:46:57,948] Trial 18 finished with value: 0.048931521673997246 and parameters: {'num_filters': 510, 'kernel_size': 4, 'pool_size': 4, 'hidden_units': 63, 'dropout_rate': 0.2937397190619352, 'learning_rate': 3.1259054639587634e-05}. Best is trial 10 with value: 0.028469223529100418.


Test Loss: 0.048931521673997246
Average Absolute Difference: 0.1795970724208122
Average Absolute Fake Difference: 0.23902514304272385


[I 2023-05-03 14:47:19,635] Trial 19 finished with value: 0.03489603102207184 and parameters: {'num_filters': 308, 'kernel_size': 5, 'pool_size': 3, 'hidden_units': 40, 'dropout_rate': 0.1485637354037996, 'learning_rate': 5.788461171859879e-05}. Best is trial 10 with value: 0.028469223529100418.


Test Loss: 0.03489603102207184
Average Absolute Difference: 0.14860170045485632
Average Absolute Fake Difference: 0.23002482814367636


In [162]:
# Get the best 5 trials
completed_trials = study.get_trials(deepcopy=False, states=[optuna.trial.TrialState.COMPLETE])
best_trials = sorted(completed_trials, key=lambda t: t.value)[:5]

# Print the best 5 trials' parameters and their respective values
for i, trial in enumerate(best_trials):
    print(f"Best trial {i + 1}:")
    print(f"  Value: {trial.value}")
    print(f"  Params: {trial.params}")

Best trial 1:
  Value: 0.028469223529100418
  Params: {'num_filters': 369, 'kernel_size': 5, 'pool_size': 4, 'hidden_units': 70, 'dropout_rate': 0.10084933148470171, 'learning_rate': 4.325122114456011e-05}
Best trial 2:
  Value: 0.028658822799722355
  Params: {'num_filters': 360, 'kernel_size': 5, 'pool_size': 4, 'hidden_units': 69, 'dropout_rate': 0.10264323081169152, 'learning_rate': 4.024276019450269e-05}
Best trial 3:
  Value: 0.028858947878082592
  Params: {'num_filters': 360, 'kernel_size': 5, 'pool_size': 4, 'hidden_units': 72, 'dropout_rate': 0.11153810108298234, 'learning_rate': 5.362206874806758e-05}
Best trial 4:
  Value: 0.031221793964505196
  Params: {'num_filters': 375, 'kernel_size': 5, 'pool_size': 4, 'hidden_units': 64, 'dropout_rate': 0.10224158128898073, 'learning_rate': 0.0001456608120918707}
Best trial 5:
  Value: 0.03176876219610373
  Params: {'num_filters': 397, 'kernel_size': 5, 'pool_size': 4, 'hidden_units': 80, 'dropout_rate': 0.24156398932131568, 'learning_r